In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



import itertools as itt
import os
import sys

import holoviews as hv
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from IPython.core.display import display

In [2]:
# dpath = '/mnt/working_storage/7364_for_minian/10_02_24/split'
# animal_ID = '7364'
# date = '10_02'
# file_dict = {
#     '/home/dprotter/testmount/Protter/cohort_2_pfc_screening/dwp/7452/2024_10_11/10_56_56/final_check/My_V4_Miniscope':
#         {'animal':7452, "date":'10_11'},
#     '/home/dprotter/testmount/Protter/cohort_2_pfc_screening/dwp/7414/2024_10_11/12_06_24/final_check/My_V4_Miniscope':
#         {'animal':7414, "date":'10_11'},
#     '/home/dprotter/testmount/Protter/cohort_2_pfc_screening/dwp/7411/2024_10_10/15_54_25/final_check/My_V4_Miniscope':
#         {'animal':7411, "date":'10_10'},
#     '/home/dprotter/testmount/Protter/cohort_2_pfc_screening/dwp/7367/2024_10_10/16_47_10/expID/My_V4_Miniscope':
#         {'animal':7367, "date":'10_10'},
#     '/home/dprotter/testmount/Protter/cohort_2_pfc_screening/dwp/7354/2024_10_11/11_22_41/expID/My_V4_Miniscope':
#         {'animal':7354, "date":'10_11'},
#     '/home/dprotter/testmount/Protter/cohort_2_pfc_screening/dwp/7354/2024_10_11/11_22_41/expID/My_V4_Miniscope':
#         {'animal':7354, "date":'10_11'},
    
# }

file_dict = {
    '/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split':
        {'animal':529, "date":'10_22'},
 
}

## import minian

The following cell loads **minian** and usually should not be modified. If you encounter an `ImportError`, check that you followed the installation instructions and that `minian_path` is pointing to the right place.

In [3]:
%%capture
# Set up Initial Basic Parameters#
minian_path = "."
sys.path.append(minian_path)
from minian.cnmf import (
    compute_AtC,
    compute_trace,
    get_noise_fft,
    smooth_sig,
    unit_merge,
    update_spatial,
    update_temporal,
    update_background,
)
from minian.initialization import (
    gmm_refine,
    initA,
    initC,
    intensity_refine,
    ks_refine,
    pnr_refine,
    seeds_init,
    seeds_merge,
)
from minian.motion_correction import apply_transform, estimate_motion
from minian.preprocessing import denoise, remove_background
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)
from minian.visualization import (
    CNMFViewer,
    VArrayViewer,
    generate_videos,
    visualize_gmm_fit,
    visualize_motion,
    visualize_preprocess,
    visualize_seeds,
    visualize_spatial_update,
    visualize_temporal_update,
    write_video,
)



n_workers = min([int(os.getenv("MINIAN_NWORKERS", 12)), 10])
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"


hv.notebook_extension("bokeh", width=100)

interactive = False

cluster = LocalCluster(
n_workers=n_workers,
memory_limit="12GB",
resources={"MEM": 1},
threads_per_worker=2,
dashboard_address=":8787",
)
annt_plugin = TaskAnnotation()
cluster.scheduler.add_plugin(annt_plugin)
client = Client(cluster)



In [4]:
for path in file_dict.keys():
    
    dpath = os.path.abspath(path)
    animal_ID = file_dict[path]['animal']
    date = file_dict[path]['date']
    
    '''_________________________________________________________'''
    minian_ds_path = os.path.join(dpath, "minian")
    intpath = os.path.join(dpath,'minian_intermediate_2')
    os.environ["MINIAN_INTERMEDIATE"] = intpath
    subset = dict(frame=slice(0, None))
    subset_mc = None
    output_size = 100
    output_modifier = 0.4
    
    param_save_minian = {
        "dpath": minian_ds_path,
        "meta_dict": dict(session=-1, animal=-2),
        "overwrite": True,
    }

    # Pre-processing Parameters#
    param_load_videos = {
        "pattern": "[0-9]+.avi$",
        #"pattern": ".*.avi$",
        "dtype": np.uint8,
        "downsample": dict(frame=1,height=1,width=1),
        "downsample_strategy": "subset",
    }
    param_denoise = {"method": "median", "ksize": 7}
    param_background_removal = {"method": "tophat", "wnd": 15}

    # Motion Correction Parameters#
    subset_mc = None
    param_estimate_motion = {"dim": "frame"}

    # Initialization Parameters#
    param_seeds_init = {
        "wnd_size": 1000,
        "method": "rolling",
        "stp_size": 500,
        "max_wnd": 15,
        "diff_thres": 3,
    }
    param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
    param_ks_refine = {"sig": 0.05}
    param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
    param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
    param_init_merge = {"thres_corr": 0.8}

    # CNMF Parameters#
    param_get_noise = {"noise_range": (0.06, 0.5)}
    param_first_spatial = {
        "dl_wnd": 10,
        "sparse_penal": 0.01,
        "size_thres": (25, None),
    }
    param_first_temporal = {
        "noise_freq": 0.06,
        "sparse_penal": 1,
        "p": 1,
        "add_lag": 20,
        "jac_thres": 0.2,
    }
    param_first_merge = {"thres_corr": 0.8}
    param_second_spatial = {
        "dl_wnd": 10,
        "sparse_penal": 0.01,
        "size_thres": (25, None),
    }
    param_second_temporal = {
        "noise_freq": 0.06,
        "sparse_penal": 1,
        "p": 1,
        "add_lag": 20,
        "jac_thres": 0.4,
    }
    '''------------------ end params -----------------------------------------------------
    ----------------------------------------------------------------------------------------
    --------------------------------------------------------------------------------------'''
    
    
    varr = load_videos(dpath, **param_load_videos)
    chk, _ = get_optimal_chk(varr, dtype=float)
    
    
    '''----------------- save ---------------------------------'''
    
    varr = save_minian(
    varr.chunk({"frame": chk["frame"], "height": -1, "width": -1}).rename("varr"),
    intpath,
    overwrite=True,
    )
    
    

    
    ##is this all the glow removal?
    varr_min = varr.min("frame").compute()
    varr_ref = varr - varr_min
    
    #denoise
    param_denoise['ksize'] = 3
    varr_ref = denoise(varr_ref, **param_denoise)
    
    #remove background
    param_background_removal['wnd'] = 15
    varr_ref = remove_background(varr_ref, **param_background_removal)
    
    "---------------------save-------------------------"
    varr_ref = save_minian(varr_ref.rename("varr_ref"), dpath=intpath, overwrite=True)
    
    
    
    
    # define center for motion correction
    subset_mc = {'height': slice(63.51875000000004, 526.71875, None),
                 'width': slice(82.68124999999998, 547.08125, None)}
    motion = estimate_motion(varr_ref.sel(subset_mc), **param_estimate_motion)
    motion = save_minian( motion.rename("motion").chunk({"frame": chk["frame"]}), **param_save_minian)
    hv.output(size=output_size)
    
    motion_out_graph = visualize_motion(motion)
    hv.save(motion_out_graph, os.path.join(dpath, f'{animal_ID} {date} motion.png'))
    
    Y = apply_transform(varr_ref, motion, fill=0)
    
    Y_fm_chk = save_minian(Y.astype(float).rename("Y_fm_chk"), intpath, overwrite=True)
    Y_hw_chk = save_minian(
    Y_fm_chk.rename("Y_hw_chk"),
    intpath,
    overwrite=True,
    chunks={"frame": -1, "height": chk["height"], "width": chk["width"]},
    )
    
    max_proj_pre = varr_ref.max("frame").compute().astype(np.float32)
    max_proj_img = Y_hw_chk.max("frame").compute()
    
    
    #create max proj image and save it
    im_opts = dict(
    frame_width=800,
    aspect=varr_ref.sizes["width"] / varr_ref.sizes["height"],
    cmap="Viridis",
    colorbar=True,
    )
    
    out =  (
    regrid(
            hv.Image(
                max_proj_pre,
                ["width", "height"],
                label="before_mc",
            ).opts(**im_opts)
        )
        + regrid(
            hv.Image(
                max_proj_img.astype(np.float32),
                ["width", "height"],
                label=f"after_mc {animal_ID} {date}",
            ).opts(**im_opts)
        )
    )

    hv.save(out, os.path.join(dpath, f'{animal_ID} {date}.png'))
    
    #write motion corrected video
    write_video(Y_fm_chk, "minian_mc.mov", dpath)
    
client.close()
cluster.close()

video list is: ['/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split/output000.avi', '/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split/output001.avi', '/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split/output002.avi', '/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split/output003.avi', '/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split/output004.avi', '/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split/output005.avi', '/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split/output006.avi', '/home/dprotter/testmount/Yurika/Meadow NAc Implants Cohort 5/529_Imaging_Timeline/split_video/split/output007.avi', '/home/dprotter/testmount/Yurika/Meadow NAc Impl